In [1]:
import numpy as np
import time
import pandas as pd

IN_COLAB = False
import sys
if IN_COLAB:
  
  scripts_dir = '/drive/My Drive/Colab Notebooks/scripts/'
  sys.path.insert(1, scripts_dir)
# from opfunu.cec_basic.cec2014_nobias import *
# from mealpy.swarm_based.PSO import BasePSO

# insert at 1, 0 is the script path (or '' in REPL)
else:
    sys.path.insert(1, 'scripts')

from geneticalgorithm import geneticalgorithm as ga
from geneticalgorithm1 import geneticalgorithm1 as ga1
from geneticalgorithmOptd import geneticalgorithmOptd as gaOptd

from PSO import BasePSO
from BBO import BaseBBO

import pandas as pd

# import benchmark_func as bf

import importlib
bf = importlib.import_module("benchmark_func")

In [2]:
dimension = 10
population_size = 15
max_iter = 50
num_runs = 5
seeds = np.random.randint(0, 1000, num_runs)
varbound=np.array([[-100, 100]]*dimension)


In [3]:
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = f'results/high_pop/Run_Results-{timestr}'
pivot_filename = "resultFile_d{}_pop{}_iter{}_runs{}--{}.xlsx"\
    .format(dimension, population_size, max_iter, num_runs, timestr)

one_args = getattr(bf, "__oneArgument__")
two_args = getattr(bf, "__twoArgument__")

In [4]:
result = {}
result["function"] = None
result["run"] = None
result["seed"] = None
result["algo"] = None
result["pop_size"] = None
result["g_opt_val"] = None
result["func_val"] = None

result["max_iter"] = None
result["best_iter"] = None

results = []


In [5]:
def createPop(dimension=2, population_size=15):
  var=np.zeros(dimension) 

  pop=np.array([np.zeros(dimension)]*population_size)
  for p in range(0, population_size):
      for i in range(0, dimension):
          var[i]=np.random.randint(varbound[i][0],\
                  varbound[i][1]+1) 
      pop[p] = var
  data = {}
  data['pop'] = pop.copy()

  # print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
  # print(data['pop'])

  return data

In [6]:
def normGa(obj_func, pop_data=None, seed = 777, terminal_value=None):
  algorithm_param = {'max_num_iteration': max_iter,\
                   'population_size':population_size,\
                   'mutation_probability':0.2,\
                   'elit_ratio': 0.1,\
                   'crossover_probability': .7,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None,\
                   'terminal_value': terminal_value}
  model=ga(function=obj_func,dimension=dimension,variable_type='real',variable_boundaries=varbound, random_seed=seed, algorithm_parameters=algorithm_param)
  model.run('GA_data.dat', data=pop_data)

  data = {}
  data['best_sol'] = model.best_variable
  data['best_fit'] = model.best_function
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = model.iterate
  return data

In [7]:
def imprvGa(obj_func, pop_data=None, seed = 777, terminal_value=None):
  algorithm_param = {'max_num_iteration': max_iter,\
                   'population_size':population_size,\
                   'mutation_probability':0.3,\
                   'elit_ratio': 0.1,\
                   'crossover_probability': .8,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None,\
                   'terminal_value': terminal_value}
  model=gaOptd(function=obj_func,dimension=dimension,variable_type='real',variable_boundaries=varbound, random_seed=seed, algorithm_parameters=algorithm_param)
  model.run('GA_data.dat', data=pop_data)

  data = {}
  data['best_sol'] = model.best_variable
  data['best_fit'] = model.best_function
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = model.iterate
  # data['best_sol'] = best_pos1
  # data['best_sol'] = best_pos1
  return data

In [8]:
def pso(obj_func, pop_data=None, seed = 777, terminal_value=None):
  lb = varbound[:, 0].tolist()
  ub = varbound[:, 1].tolist()

  verbose = False
  
  model = BasePSO(obj_func, lb, ub, verbose, max_iter, population_size, random_seed=seed, terminal_value=terminal_value)  # Remember the keyword "problem_size"
  best_sol, best_fit, list_loss1 = model.train('pso_data.dat', data=pop_data)
  data = {}
  data['best_sol'] = best_sol
  data['best_fit'] = best_fit
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = max_iter
  # data['best_sol'] = best_pos1
  # data['best_sol'] = best_pos1
  return data

In [9]:
def bbo(obj_func, pop_data=None, seed = 777, terminal_value=None):
  lb = varbound[:, 0].tolist()
  ub = varbound[:, 1].tolist()

  verbose = False

  model = BaseBBO(obj_func, lb, ub, verbose, max_iter, population_size, random_seed=seed, terminal_value=terminal_value)  # Remember the keyword "problem_size"
  best_sol, best_fit, list_loss1 = model.train('bbo_data.dat', data=pop_data)
  data = {}
  data['best_sol'] = best_sol
  data['best_fit'] = best_fit
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = max_iter
  # data['best_sol'] = best_pos1
  # data['best_sol'] = best_pos1
  return data


In [10]:
for func in two_args:
  class_ = getattr(bf, func)
  instance = class_(dimension)
  count = 1
  for seed in seeds:
    np.random.seed(seed)
    obj_func = instance.get_func_val
    pop_data = createPop(dimension, population_size)

    print("NormGA")
    res = normGa(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "NormGA"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    print("imprvGA")
    res = imprvGa(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "imprvGA"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    print("PSO")
    res = pso(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "PSO"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    print("BBO")
    res = bbo(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "BBO"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    

    count += 1


NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
impr

In [11]:
pd.DataFrame.from_dict(results, orient='columns').to_csv(f'{filename}.csv')

In [12]:
results_df = pd.DataFrame(results)


In [13]:
algos = results_df.algo.unique()
functions = results_df.function.unique()

In [14]:
describe_result = {}
describe_results = []
describe_results_df = None

count = 0
for i in range(len(algos)):
    for j in range(len(functions)):
        describe_result = {}
        new_df = results_df.loc[(results_df['function'] == functions[j]) & (results_df['algo'] == algos[i])]
        describe = new_df.describe().loc[['mean','std'], 'func_val'].tolist()
        describe_result['function'] = functions[j]
        describe_result['algo'] = algos[i]
        describe_result['mean'] = describe[0]
        describe_result['std'] = describe[1]
        describe_results.append(describe_result.copy())
    

In [15]:
func_global_opts = {}
for func in two_args:
    
    class_ = getattr(bf, func)
    instance = class_(dimension)
    func_global_opts[instance.func_name] = instance.global_optimum_solution

In [16]:
for data in describe_results:
    data['global_val'] = func_global_opts[data['function']]

In [17]:
describe_results_df = pd.DataFrame(describe_results)

In [18]:
describe_results_df.head(50)

,function,algo,mean,std,global_val
0,Ackley,NormGA,1.999270e+01,3.046231e-03,0.00000
1,Sphere,NormGA,1.429390e+03,3.765034e+02,0.00000
2,Rosenbrock,NormGA,3.754724e+07,4.051574e+07,0.00000
3,StyblinskiTang,NormGA,2.713672e+05,1.178940e+05,-391.66165
4,DeJongsF1,NormGA,1.429390e+03,3.765034e+02,0.00000
5,DeJongsF2,NormGA,3.754724e+07,4.051574e+07,0.00000
6,Ellipsoid,NormGA,1.131893e+07,9.943038e+06,0.00000
7,KTablet,NormGA,5.883433e+06,2.304441e+06,0.00000
8,WeightedSphere,NormGA,6.263213e+03,2.839655e+03,0.00000
9,HyperEllipsodic,NormGA,6.263213e+03,2.839655e+03,0.00000


In [19]:
pivot_table_df = pd.pivot_table(describe_results_df,

        index=['function', 'global_val'],

        columns=['algo'],

        values=['mean', 'std'],

        aggfunc=sum)
pivot_table_df

mean                              \
algo                                     BBO        NormGA           PSO   
function            global_val                                             
Ackley               0.00000    1.999623e+01  1.999270e+01  1.999740e+01   
DeJongsF1            0.00000    2.721960e+03  1.429390e+03  5.018148e+02   
DeJongsF2            0.00000    2.750132e+08  3.754724e+07  6.492425e+06   
Ellipsoid            0.00000    6.292645e+06  1.131893e+07  3.670553e+06   
Griewank             0.00000    6.080790e-01  3.650153e-01  1.434779e-01   
HyperEllipsodic      0.00000    1.467474e+04  6.263213e+03  4.765136e+03   
KTablet              0.00000    1.223666e+07  5.883433e+06  4.063414e+06   
Michalewicz         -1.80130   -5.426644e+00 -5.304329e+00 -2.633837e+00   
Rastrigin            0.00000    1.993570e+03  1.407270e+03  6.204728e+02   
Rosenbrock           0.00000    2.750132e+08  3.754724e+07  6.492425e+06   
Schwefel            -418.98290 -5.547185e+02 -5.253856e+02 -4.486434e+02   
Sphere               0.00000    2.721960e+03  1.429390e+03  5.018148e+02   
StyblinskiTang      -391.66165  8.254974e+05  2.713672e+05  7.720857e+04   
SumOfDifferentPower  0.00000    7.741292e+09  2.365746e+10  3.783199e+10   
WeightedSphere       0.00000    1.467474e+04  6.263213e+03  4.765136e+03   
XinSheYang           0.00000    4.230794e-02  5.979642e-02  5.502566e-01   
Zakharov             0.00000   -5.148856e+01 -8.150412e+01 -3.619677e+01   

                                                       std                \
algo                                 imprvGA           BBO        NormGA   
function            global_val                                             
Ackley               0.00000    1.997271e+01  1.082390e-03  3.046231e-03   
DeJongsF1            0.00000    5.932233e+02  7.981354e+02  3.765034e+02   
DeJongsF2            0.00000    7.607667e+06  1.361328e+08  4.051574e+07   
Ellipsoid            0.00000    3.261045e+06  6.566524e+06  9.943038e+06   
Griewank             0.00000    8.412218e-02  2.379318e-01  7.651980e-02   
HyperEllipsodic      0.00000    2.116077e+03  6.725296e+03  2.839655e+03   
KTablet              0.00000    2.917445e+06  6.500799e+06  2.304441e+06   
Michalewicz         -1.80130   -6.291533e+00  1.871474e-01  4.120170e-01   
Rastrigin            0.00000    6.872379e+02  5.112256e+02  4.256869e+02   
Rosenbrock           0.00000    7.607667e+06  1.361328e+08  4.051574e+07   
Schwefel            -418.98290 -5.766775e+02  2.562071e+01  1.727957e+01   
Sphere               0.00000    5.932233e+02  7.981354e+02  3.765034e+02   
StyblinskiTang      -391.66165  4.746243e+04  5.054188e+05  1.178940e+05   
SumOfDifferentPower  0.00000    7.304556e+08  3.871391e+09  1.468490e+10   
WeightedSphere       0.00000    2.116077e+03  6.725296e+03  2.839655e+03   
XinSheYang           0.00000    3.164701e-02  9.908457e-03  1.994203e-02   
Zakharov             0.00000   -1.282950e+01  6.726007e+01  7.351345e+01   

                                                            
algo                                     PSO       imprvGA  
function            global_val                              
Ackley               0.00000    5.940309e-04  1.692619e-02  
DeJongsF1            0.00000    1.329196e+02  1.814330e+02  
DeJongsF2            0.00000    6.192245e+06  8.327020e+06  
Ellipsoid            0.00000    3.175526e+06  1.119014e+06  
Griewank             0.00000    6.318265e-02  9.653104e-02  
HyperEllipsodic      0.00000    2.564025e+03  3.407681e+02  
KTablet              0.00000    2.895553e+06  9.070883e+05  
Michalewicz         -1.80130    2.454544e-01  8.018797e-01  
Rastrigin            0.00000    1.402617e+02  2.022316e+02  
Rosenbrock           0.00000    6.192245e+06  8.327020e+06  
Schwefel            -418.98290  2.651389e+01  2.070413e+01  
Sphere               0.00000    1.329196e+02  1.814330e+02  
StyblinskiTang      -391.66165  1.268085e+04  1.406240e+04  
SumOfDifferentPower  0

In [20]:
func_global_opts

{'Ackley': 0,
 'Sphere': 0,
 'Rosenbrock': 0,
 'StyblinskiTang': -391.66165,
 'DeJongsF1': 0,
 'DeJongsF2': 0,
 'Ellipsoid': 0.0,
 'KTablet': 0.0,
 'WeightedSphere': 0.0,
 'HyperEllipsodic': 0.0,
 'SumOfDifferentPower': 0.0,
 'Griewank': 0.0,
 'Michalewicz': -1.8013,
 'Rastrigin': 0.0,
 'Schwefel': -418.9829,
 'XinSheYang': 0.0,
 'Zakharov': 0.0}

In [21]:
pivot_table_df.keys()

MultiIndex([('mean',     'BBO'),
            ('mean',  'NormGA'),
            ('mean',     'PSO'),
            ('mean', 'imprvGA'),
            ( 'std',     'BBO'),
            ( 'std',  'NormGA'),
            ( 'std',     'PSO'),
            ( 'std', 'imprvGA')],
           names=[None, 'algo'])

In [22]:
pivot_table_df.to_excel(pivot_filename)

In [23]:
aa = "dsj{}-{}={}".format(2, 3, 4)

In [24]:
aa

'dsj2-3=4'

In [25]:
# describe_results_df.to_excel("globals.xlsx")